# Wildfire Prediction - Data Prep

## Status Updates

| Source            | Retrieval Status | Ready for Use | Work Remaining | Notes| Link |
|---------------|:---------------|:----------------|:----------------|:---------------|----------------|
|CA Wildfire Incidents|Complete | X | -- | -- | https://www.kaggle.com/datasets/ananthu017/california-wildfire-incidents-20132020 |
|FIPS CA County Codes| Complete | X | -- | -- | https://www.census.gov/library/reference/code-lists/ansi.html#cou|
|NOAA Weather (temp, pcp) | Complete | X | -- | -- | https://www.ncei.noaa.gov/pub/data/cirs/climdiv/county-readme.txt |
|Wind| N/A | -- | - Gather data | We don't have this at all yet but it seems useful. | -- | -- |
|CA Elevation| Partial | -- |  - Assess completeness<br>- Identify features<br>- Prep features | It seems like this is incomplete; HARP only provides data for specific CA regions. | https://ww2.arb.ca.gov/resources/documents/harp-digital-elevation-model-files |
|CA Vegetation | Partial -<br>In Progress (SA) | -- | - Assess completeness<br>- Identify features<br>- Prep features | I'm retrieving satellite data for the dates that preceded each fire.  Will also try to parse the link Reese provided (on right). | https://map.dfg.ca.gov/metadata/ds1020.html |
|Population and Other Demo| Partial |  | Some counties are missing |  | https://api.census.gov/data.html |

In [824]:
import json
import pandas as pd
import requests

from datetime import datetime
from io import StringIO

## Get NOAA weather data

In [825]:
# NOAA data

# from https://www.ncei.noaa.gov/pub/data/cirs/climdiv/
# README file: https://www.ncei.noaa.gov/pub/data/cirs/climdiv/county-readme.txt
# in this file CA state code is 04 (not 06)

noaa_pcp_url = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-pcpncy-v1.0.0-20240705"
noaa_tmax_url = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-tmaxcy-v1.0.0-20240705"
noaa_tmin_url = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-tmincy-v1.0.0-20240705"

In [826]:
def get_noaa_data_df(target_url):
    noaa_data = requests.get(target_url)    
    noaa_month_colnames= ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
    noaa_df = pd.read_csv(StringIO(noaa_data.text), lineterminator="\n", sep=r"\s+", header=None, names=["ID", *noaa_month_colnames], index_col=False, dtype="string")
    return noaa_df

In [827]:
pcp_data_df = get_noaa_data_df(noaa_pcp_url)
pcp_data_df.head(2)

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,01001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66
1,01001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94


In [828]:
tmax_data_df = get_noaa_data_df(noaa_tmax_url)
tmax_data_df.head(2)

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,01001271895,53.70,48.70,67.60,76.40,81.90,89.20,91.10,90.40,90.90,76.00,66.60,58.00
1,01001271896,54.20,60.80,65.30,81.60,88.50,88.20,92.00,94.50,90.80,77.20,69.90,58.70


In [829]:
tmin_data_df = get_noaa_data_df(noaa_tmin_url)
tmin_data_df.head(2)

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,01001281895,34.20,27.70,43.40,51.80,59.30,67.40,69.70,70.30,67.10,46.90,42.10,32.50
1,01001281896,34.40,37.20,42.60,57.00,65.00,67.90,71.40,71.70,65.00,52.20,46.10,35.90


In [830]:
# Concatenate NOAA dfs together
noaa_full_df = pd.concat([pcp_data_df, tmax_data_df, tmin_data_df], axis=0)

## Parse NOAA weather data

In [831]:
# Extract NOAA ID
def extract_noaa_id(df):
    df["STATE_CODE"] = df["ID"].str[:2]
    df["FIPS_CODE"] = df["ID"].str[2:5]
    df["ELEMENT_CODE"] = df["ID"].str[5:7]
    df["YEAR"] = df["ID"].str[7:]
    return df

extract_noaa_id(noaa_full_df)
noaa_full_df

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR
0,01001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66,01,001,01,1895
1,01001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94,01,001,01,1896
2,01001011897,3.27,6.63,10.94,4.35,0.81,1.57,3.96,5.02,0.87,0.75,1.84,4.38,01,001,01,1897
3,01001011898,2.33,2.07,2.60,4.56,0.54,3.13,5.80,6.02,1.51,3.21,6.66,3.91,01,001,01,1898
4,01001011899,5.80,6.94,3.35,2.22,2.93,2.31,6.80,2.90,0.63,3.02,1.98,5.25,01,001,01,1899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406905,50290282020,-22.80,-16.70,-1.60,18.40,36.30,45.90,47.10,44.80,34.90,18.30,-0.90,-3.20,50,290,28,2020
406906,50290282021,-3.30,-17.20,-6.20,11.00,33.30,46.20,49.20,43.90,31.10,20.50,-7.50,-7.60,50,290,28,2021
406907,50290282022,-12.50,-9.80,2.20,12.80,33.00,45.10,47.40,43.50,35.80,19.60,3.00,-8.50,50,290,28,2022
406908,50290282023,-4.40,-9.40,-1.70,2.10,34.60,45.10,50.80,48.00,33.00,17.20,8.80,-10.80,50,290,28,2023


In [832]:
# Map element codes to descriptive abbreviation

element_code_map = {
# element codes from docs
    "01": "pcp", # Precipitation
    "02": "tavg", # Average Temperature
    "25": "Heating Degree Days",
    "26": "Cooling Degree Days",
    "27": "tmax", # Maximum Temperature
    "28": "tmin" # Minimum Temperature
}

noaa_full_df["NOAA_ELEMENT"] = noaa_full_df["ELEMENT_CODE"].map(element_code_map).fillna(noaa_full_df["ELEMENT_CODE"])
noaa_full_df

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR,NOAA_ELEMENT
0,01001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66,01,001,01,1895,pcp
1,01001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94,01,001,01,1896,pcp
2,01001011897,3.27,6.63,10.94,4.35,0.81,1.57,3.96,5.02,0.87,0.75,1.84,4.38,01,001,01,1897,pcp
3,01001011898,2.33,2.07,2.60,4.56,0.54,3.13,5.80,6.02,1.51,3.21,6.66,3.91,01,001,01,1898,pcp
4,01001011899,5.80,6.94,3.35,2.22,2.93,2.31,6.80,2.90,0.63,3.02,1.98,5.25,01,001,01,1899,pcp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406905,50290282020,-22.80,-16.70,-1.60,18.40,36.30,45.90,47.10,44.80,34.90,18.30,-0.90,-3.20,50,290,28,2020,tmin
406906,50290282021,-3.30,-17.20,-6.20,11.00,33.30,46.20,49.20,43.90,31.10,20.50,-7.50,-7.60,50,290,28,2021,tmin
406907,50290282022,-12.50,-9.80,2.20,12.80,33.00,45.10,47.40,43.50,35.80,19.60,3.00,-8.50,50,290,28,2022,tmin
406908,50290282023,-4.40,-9.40,-1.70,2.10,34.60,45.10,50.80,48.00,33.00,17.20,8.80,-10.80,50,290,28,2023,tmin


## Create cleaned subset with just CA counties

In [833]:
noaa_ca_df = noaa_full_df[noaa_full_df["STATE_CODE"] == "04"]
noaa_ca_df

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR,NOAA_ELEMENT
20410,04001011895,8.43,2.09,1.97,1.75,1.20,0.00,0.05,0.01,0.50,0.57,1.57,1.47,04,001,01,1895,pcp
20411,04001011896,8.47,0.23,2.71,4.13,0.72,0.00,0.02,0.27,0.47,1.56,4.12,3.30,04,001,01,1896,pcp
20412,04001011897,2.66,4.30,4.11,0.56,0.14,0.21,0.00,0.02,0.15,1.82,0.97,1.54,04,001,01,1897,pcp
20413,04001011898,1.14,2.49,0.48,0.21,1.35,0.18,0.00,0.02,0.86,0.95,0.65,1.32,04,001,01,1898,pcp
20414,04001011899,4.52,0.25,7.19,0.55,0.86,0.22,0.00,0.04,0.03,4.20,3.69,2.65,04,001,01,1899,pcp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27945,04115282020,38.90,39.70,40.60,47.10,53.30,58.60,62.10,65.00,61.10,53.30,40.40,37.40,04,115,28,2020,tmin
27946,04115282021,38.50,39.40,39.50,46.70,53.40,61.40,66.40,63.40,59.00,48.50,45.70,37.70,04,115,28,2021,tmin
27947,04115282022,37.10,37.80,43.20,44.70,50.50,59.00,62.40,64.10,61.40,51.80,38.00,36.90,04,115,28,2022,tmin
27948,04115282023,36.80,35.50,37.80,44.10,52.00,56.20,63.70,63.10,56.10,50.60,42.70,41.10,04,115,28,2023,tmin


## Get county FIPS codes

In [834]:
# FIPS Codes

# from https://www.census.gov/library/reference/code-lists/ansi.html#cou
fips_url = "https://www2.census.gov/geo/docs/reference/codes2020/cou/st06_ca_cou2020.txt"

fips_data = requests.get(fips_url)
fips_df = pd.read_csv(StringIO(fips_data.text), lineterminator="\n", sep="|", dtype="string")
fips_df.head(2)

,STATE,STATEFP,COUNTYFP,COUNTYNS,COUNTYNAME,CLASSFP,FUNCSTAT
0,CA,06,001,01675839,Alameda County,H1,A
1,CA,06,003,01675840,Alpine County,H1,A


In [835]:
# Join to FIPS df
noaa_ca_counties_df = pd.merge(fips_df[["COUNTYFP", "COUNTYNAME"]], noaa_ca_df, left_on="COUNTYFP", right_on="FIPS_CODE")
noaa_ca_counties_df["COUNTYNAME"] = noaa_ca_counties_df["COUNTYNAME"].str.split().apply(lambda row: ' '.join(row[:-1]))
noaa_ca_counties_df.head(2)

,COUNTYFP,COUNTYNAME,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR,NOAA_ELEMENT
0,001,Alameda,04001011895,8.43,2.09,1.97,1.75,1.20,0.00,0.05,0.01,0.50,0.57,1.57,1.47,04,001,01,1895,pcp
1,001,Alameda,04001011896,8.47,0.23,2.71,4.13,0.72,0.00,0.02,0.27,0.47,1.56,4.12,3.30,04,001,01,1896,pcp


In [836]:
# Remove unnecessary encoding columns
noaa_ca_counties_df = noaa_ca_counties_df[["COUNTYNAME", "YEAR", 'NOAA_ELEMENT', 'JAN', 'FEB', 'MAR', 'APR', 'MAY',
       'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ID']]
noaa_ca_counties_df.head(2)

,COUNTYNAME,YEAR,NOAA_ELEMENT,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ID
0,Alameda,1895,pcp,8.43,2.09,1.97,1.75,1.20,0.00,0.05,0.01,0.50,0.57,1.57,1.47,04001011895
1,Alameda,1896,pcp,8.47,0.23,2.71,4.13,0.72,0.00,0.02,0.27,0.47,1.56,4.12,3.30,04001011896


In [837]:
noaa_long = noaa_ca_counties_df.melt(id_vars=['COUNTYNAME', 'ID', 'YEAR', 'NOAA_ELEMENT'], var_name='MONTH', value_name='NOAA_ELEMENT_VAL')
noaa_long['NOAA_ELEMENT_VAL'] = noaa_long['NOAA_ELEMENT_VAL'].astype(float)
noaa_long['MONTH_NUM'] = noaa_long['MONTH'].apply(
    lambda month_str: datetime.strptime(month_str, '%b').month)
noaa_long.head(2)

,COUNTYNAME,ID,YEAR,NOAA_ELEMENT,MONTH,NOAA_ELEMENT_VAL,MONTH_NUM
0,Alameda,04001011895,1895,pcp,JAN,8.43,1
1,Alameda,04001011896,1896,pcp,JAN,8.47,1


In [838]:
noaa_pivot_df = noaa_long.pivot_table(
    index=['COUNTYNAME', 'YEAR', 'MONTH_NUM', 'MONTH'],
    columns='NOAA_ELEMENT',
    values='NOAA_ELEMENT_VAL'
).sort_values(by=["COUNTYNAME", "YEAR", "MONTH_NUM"]).reset_index()

noaa_pivot_df["YEAR"] = noaa_pivot_df["YEAR"].astype(float)
noaa_pivot_df["MONTH_NUM"] = noaa_pivot_df["MONTH_NUM"].astype(float)

noaa_pivot_df

NOAA_ELEMENT,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin
0,Alameda,1895.0,1.0,JAN,8.43,51.5,38.8
1,Alameda,1895.0,2.0,FEB,2.09,59.3,40.7
2,Alameda,1895.0,3.0,MAR,1.97,59.8,41.4
3,Alameda,1895.0,4.0,APR,1.75,65.2,43.8
4,Alameda,1895.0,5.0,MAY,1.20,71.2,47.5
...,...,...,...,...,...,...,...
90475,Yuba,2024.0,8.0,AUG,-9.99,-99.9,-99.9
90476,Yuba,2024.0,9.0,SEP,-9.99,-99.9,-99.9
90477,Yuba,2024.0,10.0,OCT,-9.99,-99.9,-99.9
90478,Yuba,2024.0,11.0,NOV,-9.99,-99.9,-99.9


In [839]:
len(noaa_pivot_df["COUNTYNAME"].unique())

58

## DO NOT USE - INCOMPLETE
## Join Annual Census Data with Weather Data

In [840]:
# Annual data only available from 2000 onward
housing_df = pd.read_csv("../Data/Housing Data/Processed Data/CA Housing 1960-2023.csv", index_col=0)
housing_df = housing_df[["CTYNAME"] + [f"{year}" for year in range(2000, 2021)]].copy()
housing_df.columns = ["COUNTYNAME"] + list(range(2000, 2021))
housing_df["COUNTYNAME"] = housing_df["COUNTYNAME"].str.split().apply(lambda row: ' '.join(row[:-1]))
housing_df.head(2)

,COUNTYNAME,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alameda,541385,546269,550054,553821,558677,564447,569206,575668,579064,...,584528,586639,589352,592396,595760,600467,605974,615030,622922,623368
1,Alpine,1519,1540,1589,1646,1670,1688,1701,1726,1755,...,1761,1764,1765,1764,1766,1769,1770,1778,1782,1543


In [841]:
housing_long = housing_df.melt(id_vars=['COUNTYNAME'], var_name='YEAR', value_name='Housing')
housing_long.head(2)

,COUNTYNAME,YEAR,Housing
0,Alameda,2000,541385
1,Alpine,2000,1519


In [842]:
#### DO NOT USE POP DATA - MISSING COUNTIES
pop_df = pd.read_csv("../Data/Population Data/Processed Data/CA Population 1980-2023.csv", index_col=0)
len(pop_df["CTYNAME"].unique())

54

In [843]:
# Data is available earlier than 2000 but limiting to 2000 to align with housing data availability
pop_df = pd.read_csv("../Data/Population Data/Processed Data/CA Population 1980-2023.csv", index_col=0)
pop_density_df = pop_df[["CTYNAME"] + [f"{year} Population Density" for year in range(2000, 2021)]].copy()
pop_density_df.columns = ["COUNTYNAME"] + list(range(2000, 2021))
pop_density_df["COUNTYNAME"] = pop_density_df["COUNTYNAME"].str.split().apply(lambda row: ' '.join(row[:-1]))
pop_density_df.head(2)

,COUNTYNAME,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alameda,163.990499,166.212419,165.377898,164.479471,163.524601,163.052257,163.384685,164.655016,167.086076,...,173.158353,175.741319,178.657165,181.838932,184.857595,186.717000,187.741658,188.507635,188.713042,188.024319
1,Alpine,0.151125,0.149500,0.154625,0.154125,0.157875,0.152375,0.159500,0.159250,0.150250,...,0.136625,0.138875,0.141375,0.135375,0.135000,0.131625,0.139500,0.135125,0.139125,0.139875


In [844]:
pop_density_long = pop_density_df.melt(id_vars=['COUNTYNAME'], var_name='YEAR', value_name='PopDensity')
pop_density_long.head(2)

,COUNTYNAME,YEAR,PopDensity
0,Alameda,2000,163.990499
1,Alpine,2000,0.151125


In [845]:
housing_pop_df = pd.merge(pop_density_long, housing_long, on=['COUNTYNAME', 'YEAR'])
housing_pop_df["YEAR"] = housing_pop_df["YEAR"].astype(float)

noaa_housing_pop_df = pd.merge(noaa_pivot_df, housing_pop_df, on=['COUNTYNAME', 'YEAR'])
noaa_housing_pop_df

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing
0,Alameda,2000.0,1.0,JAN,5.94,57.0,42.8,163.990499,541385
1,Alameda,2000.0,2.0,FEB,7.24,57.7,44.3,163.990499,541385
2,Alameda,2000.0,3.0,MAR,1.92,63.1,43.8,163.990499,541385
3,Alameda,2000.0,4.0,APR,1.09,68.9,47.1,163.990499,541385
4,Alameda,2000.0,5.0,MAY,1.00,73.3,51.0,163.990499,541385
...,...,...,...,...,...,...,...,...,...
13603,Yuba,2020.0,8.0,AUG,0.05,95.8,65.0,11.567100,29566
13604,Yuba,2020.0,9.0,SEP,0.00,91.5,61.1,11.567100,29566
13605,Yuba,2020.0,10.0,OCT,0.00,85.4,53.3,11.567100,29566
13606,Yuba,2020.0,11.0,NOV,2.64,64.8,40.4,11.567100,29566


In [846]:
noaa_housing_pop_df.to_csv("../Data/noaa_housing_pop__DO_NOT_USE__missing_counties.csv")

## Load and filter wildfire data

In [847]:
# from Kaggle: https://www.kaggle.com/datasets/ananthu017/california-wildfire-incidents-20132020

wildfire_fpath = "../Data/California_Fire_Incidents.csv"

In [848]:
# Load wildfire data
wildfire_df = pd.read_csv(wildfire_fpath)
wildfire_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [849]:
# Retrieve year and month
wildfire_df["started_year"] = pd.to_datetime(wildfire_df["Started"], errors="coerce").dt.year
wildfire_df["started_month"] = pd.to_datetime(wildfire_df["Started"], errors="coerce").dt.month

# Handle conversion issues (4 rows)
problematic_rows = wildfire_df[wildfire_df['started_year'].isna()]
wildfire_df.loc[problematic_rows.index, 'started_year'] = problematic_rows['Started'].apply(lambda datetime_str: 
    datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ').year)
wildfire_df.loc[problematic_rows.index, 'started_month'] = problematic_rows['Started'].apply(lambda datetime_str:
    datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ').month)

In [850]:
# Filter records missing lat/lon
missing_lonlat_df = wildfire_df[(wildfire_df["Longitude"] == 0.0) | (wildfire_df["Latitude"] == 0.0)]
wildfire_df_cleaned = wildfire_df[~wildfire_df["UniqueId"].isin(missing_lonlat_df["UniqueId"])]

In [851]:
wildfire_df_cleaned["Counties"]

0          Tuolumne
1       Los Angeles
2         Riverside
3            Placer
5            Fresno
           ...     
1631      Riverside
1632         Nevada
1633           Yolo
1634      San Diego
1635      Riverside
Name: Counties, Length: 1482, dtype: object

In [852]:
# Filter non-CA counties
ca_wildfire_df = wildfire_df_cleaned[~wildfire_df_cleaned["Counties"].isin(['Mexico', 'State of Oregon', 'State of Nevada'])]
ca_wildfire_df["Counties"].unique()

array(['Tuolumne', 'Los Angeles', 'Riverside', 'Placer', 'Fresno',
       'Siskiyou', 'Humboldt', 'Tehama', 'Shasta', 'San Diego', 'Kern',
       'Ventura', 'Sonoma', 'Contra Costa', 'Butte', 'Tulare',
       'Santa Barbara', 'Mariposa', 'El Dorado', 'San Bernardino',
       'Plumas', 'Modoc', 'San Luis Obispo', 'Madera', 'Inyo', 'Napa',
       'San Benito', 'San Joaquin', 'Alameda', 'Glenn', 'Yolo',
       'Sacramento', 'Stanislaus', 'Solano', 'Merced', 'Lake', 'Monterey',
       'Mendocino', 'Lassen', 'Amador', 'Yuba', 'Nevada', 'Santa Clara',
       'Calaveras', 'San Mateo', 'Orange', 'Colusa', 'Trinity', 'Alpine',
       'Mono', 'Sutter', 'Del Norte', 'Kings', 'Sierra', 'Santa Cruz',
       'Marin'], dtype=object)

In [853]:
#### TODO: modify this if you want to export other columns
timeplace_colnames = ['started_year', 'started_month', 'Started', 'Extinguished', 'Location', 'Latitude', 'Longitude', 'Counties', 'Name']
target_colnames = ['AcresBurned', 'MajorIncident']

meta_colnames = ['UniqueId', 'CalFireIncident', 'CanonicalUrl']
impact_colnames = ['PercentContained', 'Fatalities', 'Injuries', 'StructuresDamaged', 'StructuresDestroyed', 'StructuresEvacuated', 'StructuresThreatened']
response_colnames = ['PersonnelInvolved', 'CrewsInvolved', 'WaterTenders', 'Helicopters', 'Dozers', 'Engines']
misc_colnames = ['ConditionStatement', 'FuelType', 'ControlStatement', 'Featured', 'Status', 'Final', 'Public', 'SearchDescription', 'SearchKeywords']

# Select specific columns
ca_wildfire_subset_df = ca_wildfire_df[target_colnames + timeplace_colnames]
ca_wildfire_subset_df.head(2)

,AcresBurned,MajorIncident,started_year,started_month,Started,Extinguished,Location,Latitude,Longitude,Counties,Name
0,257314.0,False,2013.0,8.0,2013-08-17T15:25:00Z,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,37.857000,-120.086000,Tuolumne,Rim Fire
1,30274.0,False,2013.0,5.0,2013-05-30T15:28:00Z,2013-06-08T18:30:00Z,Angeles National Forest,34.585595,-118.423176,Los Angeles,Powerhouse Fire


In [854]:
# Export cleaned wildfire data with subset of columns
ca_wildfire_subset_df.to_csv("../Data/ca_wildfire_cleaned.csv")

## Join cleaned wildfire data with environmental features

In [855]:
ca_wildfire_subset_df.head(2)

,AcresBurned,MajorIncident,started_year,started_month,Started,Extinguished,Location,Latitude,Longitude,Counties,Name
0,257314.0,False,2013.0,8.0,2013-08-17T15:25:00Z,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,37.857000,-120.086000,Tuolumne,Rim Fire
1,30274.0,False,2013.0,5.0,2013-05-30T15:28:00Z,2013-06-08T18:30:00Z,Angeles National Forest,34.585595,-118.423176,Los Angeles,Powerhouse Fire


In [856]:
noaa_housing_pop_df.head(2)

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing
0,Alameda,2000.0,1.0,JAN,5.94,57.0,42.8,163.990499,541385
1,Alameda,2000.0,2.0,FEB,7.24,57.7,44.3,163.990499,541385


In [857]:
# Merge cleaned NOAA and wildfires data
merged_df = pd.merge(noaa_pivot_df, ca_wildfire_subset_df, 
                     left_on=['COUNTYNAME', 'YEAR', 'MONTH_NUM'], 
                     right_on=['Counties', 'started_year', 'started_month'], how="right")

# Export data
merged_df[['COUNTYNAME', 'Counties', 'Started', 'Extinguished', 'Location', 'Latitude',
       'Longitude', 'Name',  'AcresBurned', 'MajorIncident', 'pcp', 'tmax', 'tmin',
       'YEAR', 'MONTH_NUM']].to_csv("../Data/wildfires_noaa_cleaned_formatted.csv")

## EDA (in separate files)?